In [1]:
from multidimvis_main import *

In [2]:
G = nx.read_edgelist('input/ppi_elist.txt',data=False)

### GRAPH BASED MATRICES

#### Adjacency

In [3]:
%%time

A = nx.adjacency_matrix(G)
DM_adj = A.toarray()

CPU times: user 1.27 s, sys: 231 ms, total: 1.5 s
Wall time: 1.5 s


In [4]:
df_adj = pd.DataFrame(DM_adj, columns = list(G.nodes()), index=list(G.nodes()))

In [5]:
df_adj.to_csv(r'output_csv/Adjacency_Dataframe_Human.csv', index = True)

#### Shortest Path Length

In [ ]:
%%time

d_idx_entz = {}
cc = 0
for entz in sorted(G.nodes()):
    d_idx_entz[cc] = entz
    cc += 1

Mspl = np.zeros(len(list(G.nodes())))

for n1 in range(len(list(G.nodes()))):
    vec = []
    for n2 in range(len(list(G.nodes()))):
        geneA = d_idx_entz[n1]
        geneB = d_idx_entz[n2]
        try:
            spl = nx.shortest_path_length(G,geneA,geneB)
            vec.append(spl)
        except nx.NetworkXNoPath:
            print('no path')
        
    Mspl = np.vstack((Mspl,vec))
Mspl = np.delete(Mspl, (0), axis=0)

DM_spl = Mspl 

In [ ]:
df_spl = pd.DataFrame(DM_spl, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_spl.to_csv(r'output_csv/SPL_Dataframe_Human.csv', index = True)

#### Markov: Random Walk with restart probability

In [6]:
%%time

# Restart probability
r = .8 # originally 0.8
alpha = 1.0 # indicating "randomness" 

DM_m = rnd_walk_matrix2(A, r, alpha, len(G.nodes()))

DM_m_df = pd.DataFrame(DM_m)
# DM_m_df.sum(axis=0)

DM_m_transposed = DM_m_df.T
DM_m_transposed.index = list(G.nodes())

CPU times: user 7min 47s, sys: 11.6 s, total: 7min 59s
Wall time: 2min 24s


In [8]:
%%time 

metric = "cosine" 
DM_m_new = pd.DataFrame(distance.squareform(distance.pdist(DM_m_transposed, metric)))

CPU times: user 37min 47s, sys: 9.95 s, total: 37min 57s
Wall time: 38min 4s


In [ ]:
df_m = pd.DataFrame(DM_m_new, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_m.to_csv(r'output_csv/RWR_'+metric+'_Dataframe_Human.csv', index = True)

#### -log Markov

In [ ]:
# MARKOV / RWR - LOG

#min_log = lambda t: -np.log(t)
#DM_mlog = np.array([min_log(x/max(x)) for x in DM_m])

In [ ]:
df_mlog = pd.DataFrame(DM_mlog, columns = list(G.nodes()), index=list(G.nodes()))

In [ ]:
df_mlog.to_csv(r'output_csv/RWR_log_'+metric+'_Dataframe_Human.csv', index = True)